# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [14]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.8, random_state=42)

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score` on the training set. We will use the
following parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [16]:
# Write your code here.
from sklearn.model_selection import cross_val_score
import numpy as np
my_learning_rate = [0.01, 0.1, 1, 10]
my_max_leaf_nodes = [3, 10, 30]

high_score = 0.0
for lr in my_learning_rate:
    model.set_params(classifier__learning_rate = lr)
    for ml in my_max_leaf_nodes:
        model.set_params(classifier__max_leaf_nodes = ml)
        
        cv_results = cross_val_score(estimator=model, X=data_train, y=target_train, n_jobs=2)
        if cv_results.mean() > high_score:
            high_score = cv_results.mean()
            print(f"lr = {lr} \t ml = {ml} \t score = {cv_results.mean():.2}\n")


lr = 0.01 	 ml = 3 	 score = 0.8

lr = 0.01 	 ml = 10 	 score = 0.82

lr = 0.01 	 ml = 30 	 score = 0.85

lr = 0.1 	 ml = 3 	 score = 0.85

lr = 0.1 	 ml = 10 	 score = 0.87

lr = 0.1 	 ml = 30 	 score = 0.87



Now use the test set to score the model using the best parameters that we found using cross-validation in the training set.

In [17]:
# Write your code here.
best_lr, best_ml = 0.1, 10
model.set_params(classifier__learning_rate = best_lr,
                 classifier__max_leaf_nodes = best_ml)
model.fit(data_train, target_train)
score = model.score(X=data_test, y=target_test)
print(f"accuracy is {score:.2f}")

accuracy is 0.88
